In [28]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()

True

In [29]:
stone = pd.read_excel("../data/musicbrainz.xlsx")

In [30]:
stone.head(10)

,Number,Year,Title,Artist,Type,Rating,Releases,Genre
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,38,r&b
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,47,rock
2,3,1971,Blue,Joni Mitchell,Album,4.45,18,folk
3,4,1976,Songs in the Key of Life,Stevie Wonder,Album,4.55,24,soul
4,5,1969,Abbey Road,The Beatles,Album,4.70,54,rock
5,6,1991,Nevermind,Nirvana,Album,4.45,68,grunge
6,7,1977,Rumours,Fleetwood Mac,Album,4.60,59,pop
7,8,1984,Purple Rain,Prince and The Revolution,Album + Soundtrack,4.25,36,rock
8,9,1975,Blood on the Tracks,Bob Dylan,Album,4.35,25,folk
9,10,1998,The Miseducation of Lauryn Hill,Lauryn Hill,Album,4.15,13,r&b


In [31]:
stone.isna().sum()

Number       0
Year         0
Title        0
Artist       0
Type         0
Rating      38
Releases     0
Genre        0
dtype: int64

In [32]:
stone.rename(columns={"Title": "Album"}, inplace = True)

In [33]:
stone.Album = stone.Album.replace(1999,"1999",regex=True)
stone.Album = stone.Album.replace(21,"21",regex=True)
stone.Album = stone.Album.replace(1989,"1989",regex=True)

In [34]:
stone.Artist = stone.Artist.str.strip()
stone.Album = stone.Album.str.strip()
stone.Type = stone.Type.str.strip()
stone.Genre = stone.Genre.str.strip()

In [35]:
stone["Gen"] = stone["Genre"].str.extract(r"(^([^,])+)")[0]
stone["Subgenre"] = stone["Genre"].str.extract(r",\s*(.*$)")[0]

In [38]:
stone.head(25)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,38,r&b,r&b,NaN
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,47,rock,rock,NaN
2,3,1971,Blue,Joni Mitchell,Album,4.45,18,folk,folk,NaN
3,4,1976,Songs in the Key of Life,Stevie Wonder,Album,4.55,24,soul,soul,NaN
4,5,1969,Abbey Road,The Beatles,Album,4.70,54,rock,rock,NaN
5,6,1991,Nevermind,Nirvana,Album,4.45,68,grunge,grunge,NaN
6,7,1977,Rumours,Fleetwood Mac,Album,4.60,59,pop,pop,NaN
7,8,1984,Purple Rain,Prince & The Revolution,Album + Soundtrack,4.25,36,rock,rock,NaN
8,9,1975,Blood on the Tracks,Bob Dylan,Album,4.35,25,folk,folk,NaN
9,10,1998,The Miseducation of Lauryn Hill,Lauryn Hill,Album,4.15,13,r&b,r&b,NaN


In [39]:
stone.Album = stone.Album.str.replace(r'(Metallica...The Black Album..)',"Metallica",regex=True)
stone.Album = stone.Album.str.replace(r'(Eagles...st album.)',"Eagles",regex=True)
stone.Artist = stone.Artist.str.replace("Neil Young with Crazy Horse","Neil Young & Crazy Horse",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Bob Dylan...The Band)","Bob Dylan And The Band",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Prince\s*.*)","Prince & The Revolution",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Rufus...Chaka Khan)","Rufus",regex=True)
stone.Album = stone.Album.str.replace(r'(Proud Mary.*)',"Proud Mary",regex=True)
stone.Artist = stone.Artist.str.replace(r'(.*The Velvet Underground.*)',"The Velvet Underground",regex= True)

In [41]:
stone[stone["Artist"] == "The Beatles" ]

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre
4,5,1969,Abbey Road,The Beatles,Album,4.70,54,rock,rock,NaN
10,11,1966,Revolver,The Beatles,Album,4.65,39,rock,rock,NaN
23,24,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Album,4.55,60,rock,rock,NaN
28,29,1968,The Beatles (“The White Album”),The Beatles,Album,4.40,50,rock,rock,NaN
34,35,1965,Rubber Soul,The Beatles,Album,4.45,44,rock,rock,NaN
196,197,1964,Meet The Beatles!,The Beatles,Album,4.60,32,rock,rock,NaN
262,263,1964,A Hard Day's Night,The Beatles,Album + Soundtrack,4.20,74,rock,rock,NaN
265,266,1965,Help!,The Beatles,Album + Soundtrack,4.05,84,rock,rock,NaN
341,342,1970,Let It Be,The Beatles,Album,4.00,47,rock,rock,NaN


In [42]:
stone["Alb_url"] = stone["Album"]
stone["Art_url"] = stone["Artist"]

In [43]:
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Beatles\s..The White Album..)',"The Beatles (Remastered)",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Band...The Brown Album..)',"The Band",regex=True)

In [44]:
stone["Alb_url"] = stone["Alb_url"].str.replace("'","%27",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("&","%26",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace(".","%2e",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("/","%2f",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("#","%23",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace(r"(\s+)","%20",regex=True)

stone["Art_url"] = stone["Art_url"].str.replace("'","%27",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("&","%26",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace(".","%2e",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("/","%2f",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("#","%23",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace(r"(\s+)","%20",regex=True)

In [45]:
stone.head(25)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre,Alb_url,Art_url
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,38,r&b,r&b,NaN,What%27s%20Going%20On,Marvin%20Gaye
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,47,rock,rock,NaN,Pet%20Sounds,The%20Beach%20Boys
2,3,1971,Blue,Joni Mitchell,Album,4.45,18,folk,folk,NaN,Blue,Joni%20Mitchell
3,4,1976,Songs in the Key of Life,Stevie Wonder,Album,4.55,24,soul,soul,NaN,Songs%20in%20the%20Key%20of%20Life,Stevie%20Wonder
4,5,1969,Abbey Road,The Beatles,Album,4.70,54,rock,rock,NaN,Abbey%20Road,The%20Beatles
5,6,1991,Nevermind,Nirvana,Album,4.45,68,grunge,grunge,NaN,Nevermind,Nirvana
6,7,1977,Rumours,Fleetwood Mac,Album,4.60,59,pop,pop,NaN,Rumours,Fleetwood%20Mac
7,8,1984,Purple Rain,Prince & The Revolution,Album + Soundtrack,4.25,36,rock,rock,NaN,Purple%20Rain,Prince%20%26%20The%20Revolution
8,9,1975,Blood on the Tracks,Bob Dylan,Album,4.35,25,folk,folk,NaN,Blood%20on%20the%20Tracks,Bob%20Dylan
9,10,1998,The Miseducation of Lauryn Hill,Lauryn Hill,Album,4.15,13,r&b,r&b,NaN,The%20Miseducation%20of%20Lauryn%20Hill,Lauryn%20Hill


In [46]:
stone.to_csv("../data/stone.csv")